In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install deepface

In [ ]:
import cv2
from deepface import DeepFace
from google.colab.patches import cv2_imshow
import time
import os
import glob
import requests
import json
from collections import Counter, defaultdict  # for counting and storing confidence scores

# path to folder where videos get uploaded to from the front-end
upload_folder = "/content/drive/MyDrive/EmotionalRec_Uploads"

# API endpoints for recommender
RECOMMENDER_API_URL = "x/colab_callback"  # Replace with ngrok / hosted route
TOKEN_API_URL = "x/token"  # Replace with actual ngrok / hosted route

# function to check if the recommender server is reachable
def check_server_status():
    try:
        response = requests.get(RECOMMENDER_API_URL.replace("/colab_callback", "/"))
        if response.status_code == 200:
            print("✅ Recommender API is reachable.")
            return True
        else:
            print(f"⚠️ Recommender API unreachable. Status: {response.status_code}")
            return False
    except Exception as e:
        print(f"❌ Error reaching Recommender API: {e}")
        return False

#  fetching the latest spotify access token dynamically
def get_access_token():
    try:
        response = requests.get(TOKEN_API_URL)
        if response.status_code == 200:
            return response.json().get("access_token")
        else:
            print(f"⚠️ Failed to fetch access token. Status: {response.status_code}")
            return None
    except Exception as e:
        print(f"❌ Error fetching token: {e}")
        return None

# check if server is reachable before proceeding
if not check_server_status():
    print("❌ Server is down. Exiting...")
    exit()

# main function to process a single video file
def process_video(video_path):
    print(f"📂 Found video to process: {video_path}")

    # open the video file using opencv
    cap = cv2.VideoCapture(video_path)

    # check to see if OpenCV opened the video successfully
    if not cap.isOpened():
        print("Error: The video file cannot be opened...")
        return
    else:
        print("Processing video...")

    frame_count = 0
    total_start_time = time.time()  # Start tracking total video processing time
    detected_emotions = []  # Store detected emotions in an array
    confidence_scores = defaultdict(float)  # Store confidence scores (how confident is the detection for each emotion)

    # process the video frame by frame
    while cap.isOpened():
        frame_start_time = time.time()  # Track time for specific frame
        ret, frame = cap.read()  # ret is a boolean check if frame is being read successfully

        if not ret:
            break  # Exit when video ends

        frame_count += 1  # count processed frames

        # skip processing unless it's every 5th frame
        if frame_count % 5 != 0:
            continue

        try:
            result = DeepFace.analyze(
                img_path=frame,
                actions=["emotion"],
                detector_backend="retinaface",  # DeepFace emotion detection model options:(mtcnn, opencv & dlib)
                enforce_detection=False  # Returns 'neutral' if no face detected rather than error if no face found.
            )

            # get a dominant emotion and its confidence score
            dominant_emotion = result[0]['dominant_emotion']  # Most confident emotion
            confidence_score = result[0]['emotion'][dominant_emotion]  # Confidence percentage

            detected_emotions.append(dominant_emotion)  # Store detected emotion
            confidence_scores[dominant_emotion] += confidence_score  # Accumulate confidence

            print(f"🧠 Frame {frame_count}: Detected Emotion - {dominant_emotion} ({confidence_score:.2f}%)")  # Log detected emotion

            # Draw emotion text on frame
            cv2.putText(frame, f"Emotion: {dominant_emotion}", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Show frame (displays analysis frame by frame)
            cv2_imshow(frame)

        except Exception as e:
            print(f"⚠️ Error processing frame {frame_count}:", e)

        # Measure processing time for this frame
        frame_end_time = time.time()
        frame_processing_time = frame_end_time - frame_start_time
        print(f"⏳ Time taken for frame {frame_count}: {frame_processing_time:.4f} seconds")

    # Releasing resources
    cap.release()

    # Calculate & print performance stats
    total_end_time = time.time()
    total_processing_time = total_end_time - total_start_time
    print(f"✅ Total Video Processing Time: {total_processing_time:.2f} seconds")
    print(f"⚡ Average FPS: {frame_count / total_processing_time:.2f}")

    # determine the most common detected emotion
    if detected_emotions:
        most_common_emotion = Counter(detected_emotions).most_common(1)[0][0]
        highest_confidence_emotion = max(confidence_scores, key=confidence_scores.get)

        # final emotion choice (can be changed to use the confidence-based approach)
        final_detected_emotion = most_common_emotion

        print(f"🎭 Most Common Emotion Detected (Occurrence-Based): {most_common_emotion}")
        print(f"🔝 Most Confident Emotion Detected (Confidence-Based): {highest_confidence_emotion}")
        print(f"🚀 Sending emotion to recommender...")

        # fetch a fresh token BEFORE sending, in case user logged out
        access_token = get_access_token()
        payload = {"emotion": final_detected_emotion}
        if access_token:
            payload["access_token"] = access_token  # optional inclusion only if available

        # send the detected emotion to recommender API
        try:
            response = requests.post(RECOMMENDER_API_URL, json=payload)
            if response.status_code == 200:
                response_data = response.json()
                print("✅ Recommender API response:", response_data)

                if response_data.get("playlist_created"):
                    print(f"🎵 Playlist Created: {response_data['playlist_created']['playlist_url']}")

                # delete the video after successful API call
                try:
                    os.remove(video_path)
                    print(f"🗑️ Deleted video from Drive: {video_path}")
                except Exception as e:
                    print(f"⚠️ Could not delete video file: {e}")

            else:
                print(f"⚠️ Failed to send emotion to API. Status Code: {response.status_code}")

        except Exception as e:
            print(f"❌ Error sending emotion to API: {e}")

# polling loop to detect new uploads automatically
print("🔄 Waiting for new uploads...")

last_seen = None

while True:
    video_files = glob.glob(os.path.join(upload_folder, "*.mp4"))
    video_files.sort()

    if video_files:
        latest = video_files[-1]
        if latest != last_seen:
            requests.post("x/status_update", json={"status": "processing_started"})

            print(f"\n📂 New video detected: {latest}")
            last_seen = latest
            process_video(latest)
        else:
            print("⏳ No new video found.")
    else:
        print("📭 Upload folder is empty.")

    time.sleep(5)  # checks every 5 seconds
